In [7]:
import pandas as pd
import statistics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
import tsfresh.feature_extraction.feature_calculators as fc
from scipy.io import wavfile
import rpy2.robjects as robjects
import numpy as np
import os
import re
import wave
import pickle
import librosa
import sounddevice as sd
import time
import serial
import matplotlib.pyplot as plt
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import py2rpy
from rpy2.robjects import r, pandas2ri



In [16]:
# Load the saved model
rds_file = "rf_fit.rds"
rf_fit = robjects.r['readRDS'](rds_file)
# Convert the R object to a pandas DataFrame
rf_fit_py = pandas2ri.rpy2py(rf_fit)


In [17]:
def extract_features(data):
    # Load audio file
    y, sr = librosa.load(data, sr=None)

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, fmin=40)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    # Concatenate features
    features = np.concatenate((chroma_stft, rmse, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_rolloff, mfcc), axis=0)

    return features

new_features = extract_features("L3/BBB_1.wav")

In [18]:
# Make a prediction using the loaded classifier
new_prediction = robjects.r['predict'](rf_fit, pd.DataFrame(new_features).T)

# Print the predicted class
print(new_prediction[0])

R[write to console]: predict.randomForest(list(call = randomForest(x = X, y = y, ntree = 500, 에서 다음과 같은 에러가 발생했습니다:
  variables in the training data missing in newdata



RRuntimeError: predict.randomForest(list(call = randomForest(x = X, y = y, ntree = 500, 에서 다음과 같은 에러가 발생했습니다:
  variables in the training data missing in newdata


Streaming RF classifier

In [240]:
def read_arduino(ser,inputBufferSize):
#    data = ser.readline(inputBufferSize)
    data = ser.read(inputBufferSize)
    out =[(int(data[i])) for i in range(0,len(data))]
    return out

def process_data(data):
    data_in = np.array(data)
    result = []
    i = 1
    while i < len(data_in)-1:
        if data_in[i] > 127:
            # Found beginning of frame
            # Extract one sample from 2 bytes
            intout = (np.bitwise_and(data_in[i],127))*128
            i = i + 1
            intout = intout + data_in[i]
            result = np.append(result,intout)
        i=i+1
    return result

In [242]:
# Read example data
baudrate = 230400
cport = 'COM4'  # set the correct port before you run it
#cport = '/dev/tty.usbmodem141101'  # set the correct port before run it
ser = serial.Serial(port=cport, baudrate=baudrate)    
# take example data
inputBufferSize = 10000 # 20000 = 1 second
ser.timeout = inputBufferSize/10000.0  # set read timeout
ser.set_buffer_size(rx_size = inputBufferSize)

AttributeError: module 'serial' has no attribute 'Serial'

In [241]:
data = read_arduino(ser,inputBufferSize)

NameError: name 'ser' is not defined

In [234]:
# Define a function to extract features from audio data
def extract_features(data):
    # Load audio file
    y, sr = librosa.load(data, sr=None)

    # Extract features
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    # Concatenate features
    features = np.concatenate((chroma_stft, rmse, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_rolloff, mfcc), axis=0)

    return features

In [233]:
# Define a function to classify audio data
def classify(data):
    # Extract features from the audio data
    features = extract_features(data)
    # Reshape the features to match the input shape of the classifier
    new_features = features.reshape(1, -1)
    # Make a prediction using the loaded classifier
    new_prediction = rf_fit.predict(new_features)
    # Return the predicted class
    return new_prediction

In [ ]:
# take continuous data stream 
inputBufferSize = 10000 # keep between 2000-20000
ser.timeout = inputBufferSize/20000.0  # set read timeout, 20000 is one second
ser.set_buffer_size(rx_size=inputBufferSize)

In [ ]:
# Plot the data as it is being read
%matplotlib notebook

total_time = 20.0; # time in seconds [[1 s = 20000 buffer size]]
max_time = 10.0; # time plotted in window [s]
N_loops = 20000.0/inputBufferSize*total_time

T_acquire = inputBufferSize/20000.0    # length of time that data is acquired for 
N_max_loops = max_time/T_acquire    # total number of loops to cover desire time window

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
plt.ion()
fig.show()
fig.canvas.draw()

In [ ]:
for k in range(0,int(N_loops)):
    data = read_arduino(ser,inputBufferSize)
    data_temp = process_data(data)
    
    if k <= N_max_loops:
        if k==0:
            data_plot = data_temp
        else:
            data_plot = np.append(data_temp,data_plot)
        t = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot).size)
    else:
        data_plot = np.roll(data_plot,len(data_temp))
        data_plot[0:len(data_temp)] = data_temp
    t = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot).size)

    # Classify the audio data
    prediction = classify(data)
    # Print the predicted class
    print(prediction)
    
    ax1.clear()
    ax1.set_xlim(0, max_time)
    plt.xlabel('time [s]')
    ax1.plot(t,data_plot)
    fig.canvas.draw()    
    plt.show()